In [5]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
'''import seaborn as sns'''
from time import time

In [11]:
titanic_train = pd.read_csv("/home/w/Data Mining/HW1/train.csv", dtype={"Age": np.float64},)
titanic_test = pd.read_csv("/home/w/Data Mining/HW1/test.csv", dtype={"Age": np.float64},)

In [14]:
train_set = titanic_train.drop("Survived", axis = 1)
df_combo = pd.concat((train_set, titanic_test), axis = 0, ignore_index = True)

df_combo["Embarked"] = df_combo["Embarked"].fillna("C")

Title_list = pd.DataFrame(index = df_combo.index, columns = ["Title"])
Surname_list = pd.DataFrame(index = df_combo.index, columns = ["Surname"])
Name_list = list(df_combo.Name)
NL_1 = [elem.split("\n") for elem in Name_list]
ctr = 0
for j in NL_1:
    FullName = j[0]
    FullName = FullName.split(",")
    Surname_list.loc[ctr,"Surname"] = FullName[0]
    FullName = FullName.pop(1)
    FullName = FullName.split(".")
    FullName = FullName.pop(0)
    FullName = FullName.replace(" ", "")
    Title_list.loc[ctr, "Title"] = str(FullName)
    ctr = ctr + 1


#Title and Surname Extraction

Title_Dictionary = {
"Capt": "Officer",
"Col": "Officer",
"Major": "Officer",
"Jonkheer": "Sir",
"Don": "Sir",
"Sir" : "Sir",
"Dr": "Dr",
"Rev": "Rev",
"theCountess": "Lady",
"Dona": "Lady",
"Mme": "Mrs",
"Mlle": "Miss",
"Ms": "Mrs",
"Mr" : "Mr",
"Mrs" : "Mrs",
"Miss" : "Miss",
"Master" : "Master",
"Lady" : "Lady"
}    
    
def Title_Label(s):
    return Title_Dictionary[s]

df_combo["Title"] = Title_list["Title"].apply(Title_Label)
    

Surname_Fam = pd.concat([Surname_list, df_combo[["SibSp", "Parch"]]], axis = 1)
Surname_Fam["Fam"] = Surname_Fam.Parch + Surname_Fam.SibSp + 1

Surname_Fam = Surname_Fam.drop(["SibSp", "Parch"], axis = 1)

df_combo = pd.concat([df_combo, Surname_Fam], axis = 1)


Cabin_List = df_combo.loc[:,["Cabin"]]
Cabin_List = Cabin_List.fillna("UNK")
Cabin_Code = []
for j in Cabin_List.Cabin:
    Cabin_Code.append(j[0])
Cabin_Code = pd.DataFrame({"Deck" : Cabin_Code})
df_combo = pd.concat([df_combo, Cabin_Code], axis = 1)


def Fam_label(s):
    if (s >= 2) & (s <= 4):
        return 2
    elif ((s > 4) & (s <= 7)) | (s == 1):
        return 1
    elif (s > 7):
        return 0

df_combo["Fam"] = df_combo.loc[:,"Fam"].apply(Fam_label)


def Title_label(s):
    if (s == "Sir") | (s == "Lady"):
        return "Royalty"
    elif (s == "Dr") | (s == "Officer") | (s == "Rev"):
        return "Officer"
    else:
        return s
        
df_combo["Title"] = df_combo.loc[:,"Title"].apply(Title_label)     


def tix_clean(j):
    j = j.replace(".", "")
    j = j.replace("/", "")
    j = j.replace(" ", "")
    return j
    
df_combo[["Ticket"]] = df_combo.loc[:,"Ticket"].apply(tix_clean)

Ticket_count = dict(df_combo.Ticket.value_counts())

def Tix_ct(y):
    return Ticket_count[y]

df_combo["TicketGrp"] = df_combo.Ticket.apply(Tix_ct)
def Tix_label(s):
    if (s >= 2) & (s <= 4):
        return 2
    elif ((s > 4) & (s <= 8)) | (s == 1):
        return 1
    elif (s > 8):
        return 0

df_combo["TicketGrp"] = df_combo.loc[:,"TicketGrp"].apply(Tix_label)   

df_combo.drop(["PassengerId", "Name", "Ticket", "Surname", "Cabin", "Parch", "SibSp"], axis=1, inplace = True)

## Filling missing Age data

mask_Age = df_combo.Age.notnull()
Age_Sex_Title_Pclass = df_combo.loc[mask_Age, ["Age", "Title", "Sex", "Pclass"]]
Filler_Ages = Age_Sex_Title_Pclass.groupby(by = ["Title", "Pclass", "Sex"]).median()
Filler_Ages = Filler_Ages.Age.unstack(level = -1).unstack(level = -1)


mask_Age = df_combo.Age.isnull()
Age_Sex_Title_Pclass_missing = df_combo.loc[mask_Age, ["Title", "Sex", "Pclass"]]

def Age_filler(row):
    if row.Sex == "female":
        age = Filler_Ages.female.loc[row["Title"], row["Pclass"]]
        return age
    
    elif row.Sex == "male":
        age = Filler_Ages.male.loc[row["Title"], row["Pclass"]]
        return age
    
Age_Sex_Title_Pclass_missing["Age"]  = Age_Sex_Title_Pclass_missing.apply(Age_filler, axis = 1)   

df_combo["Age"] = pd.concat([Age_Sex_Title_Pclass["Age"], Age_Sex_Title_Pclass_missing["Age"]])    

dumdum = (df_combo.Embarked == "S") & (df_combo.Pclass == 3)
df_combo.fillna(df_combo[dumdum].Fare.median(), inplace = True)


#df_combo["Age"] = (df_combo["Age"] - df_combo["Age"].mean())/df_combo["Age"].std()
#df_combo["Fare"] = (df_combo["Fare"] - df_combo["Fare"].mean())/df_combo["Fare"].std()
    
#### OHE encoding nominal categorical features ###
df_combo = pd.get_dummies(df_combo)


df_train = df_combo.loc[0:len(titanic_train["Survived"])-1]
df_test = df_combo.loc[len(titanic_train["Survived"]):]
total_number_param = len(df_train.columns)
df_target = titanic_train.Survived

from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

In [29]:
from sklearn.pipeline import make_pipeline

select = SelectKBest(k = 20)
clf = RandomForestClassifier(warm_start = True, oob_score = True,random_state = 50,
                                  n_estimators = 100,max_depth = 12, 
                                  max_features = 'auto')
pipeline = make_pipeline(select, clf)               
 
#select.fit(df_train, df_target)

pipeline.fit(df_train, df_target)
predictions = pipeline.predict(df_train)
predict_proba = pipeline.predict_proba(df_train)[:,1]

In [32]:
cv_score = cross_validation.cross_val_score(pipeline, df_train, df_target, cv= 10)
print("Accuracy : %.4g" % metrics.accuracy_score(df_target.values, predictions))
print("AUC Score (Train): %f" % metrics.roc_auc_score(df_target, predict_proba))
print("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score), np.std(cv_score), 
np.min(cv_score),
np.max(cv_score)))

Accuracy : 0.9618
AUC Score (Train): 0.989271
CV Score : Mean - 0.8361767 | Std - 0.03752349 | Min - 0.752809 | Max - 0.8876404


In [31]:
final_pred = pipeline.predict(df_test)
submission = pd.DataFrame({"PassengerId": titanic_test["PassengerId"], "Survived": final_pred })
submission.to_csv("/home/w/Desktop/tmp/RandomForest_v2.csv", index=False) 

In [33]:
SVM

NameError: name 'SVM' is not defined

In [47]:
from sklearn.svm import SVC
svm_clf = SVC(C = 30, gamma = 0.01)
svm_clf.fit(df_train, df_target)

svm_output = svm_clf.predict(df_test)

In [48]:
from sklearn.model_selection import cross_val_score, KFold
svm_score = cross_val_score(clf, df_train, df_target, cv=10)
print svm_score

[ 0.81111111  0.85555556  0.76404494  0.83146067  0.8988764   0.84269663
  0.84269663  0.82022472  0.82022472  0.875     ]


In [49]:
submission = pd.DataFrame({"PassengerId": titanic_test["PassengerId"], "Survived": svm_output })
submission.to_csv("/home/w/Desktop/tmp/SVMv1.csv", index=False) 

In [51]:
df_train

,Pclass,Age,Fare,Fam,TicketGrp,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,...,Title_Royalty,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_U
0,3,22.0,7.2500,2,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,1,38.0,71.2833,2,2,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
2,3,26.0,7.9250,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,1,35.0,53.1000,2,2,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,3,35.0,8.0500,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
5,3,26.0,8.4583,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
6,1,54.0,51.8625,1,2,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
7,3,2.0,21.0750,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
8,3,27.0,11.1333,2,2,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
9,2,14.0,30.0708,2,2,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
